*Исходный нотубук с заданием и ссылка на данные: https://github.com/ton4eg/coursera_pa*  

## 5.2 Programming Assignment - Анализ Изображений
В этот ноутбуке будет выполнено задание второй недели 5-го курса специализации "Машинное обучение и анализ данных" с использованием библиотеки Keras. Перед запуском ноутбука убедитесь, что в папке с ноутбуком лежат файлы для обучение и теста в папках `train` и `test`, а также файлы с метками картинок и имена классов: `results.txt` и `class_names.txt`. Подробнее про предобученные модели в Keras тут: https://keras.io/applications/  
> **Обратите внимание!** Полученные ответы отличаются от ответов, полученных мной используя только tensorflow, как предлагается в задании. Этот ноутбук - только пример того, как можно выполнить это же задание, но с использованием других инструментов и проанализировать результаты, взглянув на картинки.

### Установка Keras
Установаить Keras можно с помощью комманды:  

`conda install -c conda-forge keras`  

Ссылка на иснтрукции по установке: https://keras.io/#installation  

### Установка ipywidgets
Чтобы в этом ноутбуке работал выбор картинок для анализа результатов нужно установить и включить ipywidgets:

1. `conda install -c conda-forge ipywidgets`
2. `jupyter nbextension enable --py widgetsnbextension`

Больше информации тут: https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.models import Model

import glob
import os
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact
%matplotlib inline

In [ ]:
# Load and preprocess image from path.
def load_preprocess(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [ ]:
# Load correct labels for the dataset as dict.
def load_labels(fname):
    labels = {}
    with open(fname) as f:
        for line in f:
            fname, class_id = line.strip().split()
            labels[fname] = class_id
    return labels


# Loas class names for labels as dict.
def load_class_names(fname):
    i = 1
    class_names = {}
    with open(fname) as f:
        for car_name in f:
            class_names[i] = car_name
            i += 1
    return class_names


# Get list of input files from folder and list of labels for these files.
def get_inputs(folder, labels):
    X = glob.glob(folder + '/*.jpg')
    Y = [labels[os.path.basename(x)] for x in X]
    return X, Y

In [ ]:
# Get lists of train and test files and labels
labels = load_labels('results.txt')
train, labels_train = get_inputs('train', labels)
test, labels_test = get_inputs('test', labels)

# Get dict of classes descriptions.
class_names = load_class_names('class_names.txt')

In [ ]:
# Base VGG16 model with all layers
base_model = VGG16(weights='imagenet')

## Задание 1.

Для начала нужно запустить готовую модель `vgg16`, предобученную на `imagenet`. Модель обучена с помощью `caffe` и сконвертирована в формат `tensorflow` - `vgg16_weights.npz`. Скрипт, иллюстрирующий применение этой модели к изображению, возвращает топ-5 классов из `imagenet` и уверенность в этих классах.

**Задание:** Загрузите уверенность для первого класса для изображения `train/00002.jpg` с точностью до 1 знака после запятой в файл с ответом.

In [ ]:
@interact(img_path=train)
def classify_img(img_path):
    # Load and preprocess image.
    x = load_preprocess(img_path)
    
    # Get label and class name.
    y = labels_train[train.index(img_path)]
    y_name = class_names[int(y)]
    
    # Get model predictions for the image.
    preds = base_model.predict(x)
    
    # Decode the results into a list of tuples (class, description, probability).
    preds_decoded = decode_predictions(preds, top=5)[0]
    
    print('Predicted:')
    for _, label, p in preds_decoded:
        print('{:<20}{:.4f}'.format(label, p))
        
    plt.imshow(image.load_img(img_path))
    plt.title(y + ': ' + y_name)
    plt.show()

## Задание 2

Научитесь извлекать `fc2` слой. Возьмите за основу код `process_image`, и модифицируйте, чтобы вместо последнего слоя извлекались выходы `fc2`.

**Задание:** Посчитайте `fc2` для картинки `train/00002.jpg`.  Запишите первые 20 компонент.

In [ ]:
# Model with output of 'fc2' layer of the base model.
model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

In [ ]:
# Load and preprocess image.
x = load_preprocess('train/00002.jpg')

img_fc2_out = model.predict(x)
print(img_fc2_out[:, :20])

## Задание 3

Теперь необходимо дообучить классификатор на нашей базе. В качестве бейзлайна предлагается воспользоваться классификатором `svm` из пакета `sklearn`.

- Модифицировать функцию `get_features` и добавить возможность вычислять `fc2`. (Аналогично второму заданию).
- Применить `get_feautures`, чтобы получить `X_test` и `Y_test`.
- Воспользоваться классификатором `SVC` с `random_state=0`.

> **Важно!** Если вам не удалось поставить `tensorflow`, то необходимо вместо использования функции `get_features`, загрузить предпосчитанные `X`, `Y`, `X_test`, `Y_test` из архива: https://yadi.sk/d/RzMOK8Fjvs6Ln и воспользоваться функцией `np.load` для их загрузки, а после этого два последних пункта.

**Задание:** Сколько правильных ответов получается на валидационной выборке из папки `test`? Запишите в файл.

In [ ]:
# Get outputs of the model for list of images.
def get_features(img_paths, model):
    X = np.zeros((len(img_paths), 4096))
    for i, img_path in enumerate(img_paths):
        print(img_path)
        x = load_preprocess(img_path)
        X[i, :] = model.predict(x).squeeze()
    return X

In [ ]:
X = get_features(train, model)
X_test = get_features(test, model)

In [ ]:
from sklearn.svm import SVC
clf = SVC(random_state=0)

clf.fit(X, labels_train)
labels_predicted = clf.predict(X_test)
print('Count of correct predictions: %d of %d' % (np.sum(labels_predicted == labels_test), X_test.shape[0]))

In [ ]:
@interact(img_path=test)
def classify_car(img_path):
    # Index of the current image in test, labels_test and labels_predicted arrays.
    i = test.index(img_path)
    
    # Get true car model
    y_true = labels_test[i]
    y_name_true = class_names[int(y_true)]
    
    # Get predicted car model
    y_pred = labels_predicted[i]
    y_name_pred = class_names[int(y_pred)]
    
    print('{:<20}{}'.format('True', y_name_true))
    print('{:<20}{}'.format('Predicted', y_name_pred))
        
    plt.imshow(image.load_img(img_path))
    plt.show()